![title](https://image.ibb.co/erDntK/logo2018.png)

---

# [Class Exercise] Convolutional Neural Network Case Studies

# 1. Load CIFAR-10 Dataset

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, classification_report

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Dropout, ZeroPadding2D
from keras import Model
from keras.layers import Input
from keras.regularizers import l2

from keras.utils import np_utils
from keras import backend as K

In [ ]:

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'forse', 'ship', 'truck']

num_classes = 10
input_shape = [28, 28]

X_val = X_train[-1000:,:]
y_val = y_train[-1000:]

X_train = X_train[:-1000, :]
y_train = y_train[:-1000]

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image

print('X_train.shape =',X_train.shape)
print('X_val.shape   =',X_val.shape)
print('X_test.shape  =',X_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train.shape =',y_train.shape)
print('y_val.shape   =',y_val.shape)
print('y_test.shape  =',y_test.shape)

data = (X_train, y_train, X_val, y_val, X_test, y_test)

In [ ]:
def neuralnet():    
    return Sequential([
        Flatten(input_shape=[32, 32, 3]),
        Dense(200, activation = 'relu'),
        Dense(120, activation = 'relu'),
        Dense(84, activation = 'relu'),
        Dense(num_classes, activation = 'softmax')
    ])

In [ ]:
def try_model(model_name, data, epochs = 2, batch_size = 256 ):
    
    myModel = model_name()
    myModel.summary()
    myModel.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    X_train, y_train, X_val, y_val, X_test, y_test = data
    
    myModel.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                validation_data=(X_val, y_val), verbose=2)
    
    scores = myModel.evaluate(X_train, y_train, verbose=2)
    print('Model Train',myModel.metrics_names[1],': %.2f%%' %(scores[1] * 100))

    scores = myModel.evaluate(X_val, y_val, verbose=2)
    print('Model Val',myModel.metrics_names[1],': %.2f%%' %(scores[1] * 100))
    
    y_pred = myModel.predict(X_test)
    print('Accuracy = %.2f%%' % (accuracy_score(y_test.argmax(1), y_pred.argmax(1))*100))
    
    return myModel, y_pred

In [ ]:
myModel, y_pred = try_model(neuralnet, data)

---
# LeNet-5 [2012]

![LeNet5](https://www.researchgate.net/profile/Vladimir_Golovko3/publication/313808170/figure/fig3/AS:552880910618630@1508828489678/Architecture-of-LeNet-5.png)

In [ ]:
def lenet5():
    return Sequential([
        Conv2D(6, (5, 5), input_shape=[32, 32, 3], activation='relu'),
        AveragePooling2D(pool_size=(2, 2)),
        Conv2D(16, (5, 5), activation='relu'),
        AveragePooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(120, activation = 'relu'),
        Dense(84, activation = 'relu'),
        Dense(num_classes, activation = 'softmax')
    ])

In [ ]:
myModel, y_pred = try_model(lenet5, data)

---
# AlexNet [2012]

The neural network developed by Krizhevsky, Sutskever, and Hinton in 2012 was the coming out party for CNNs in the computer vision community. This was the first time a model performed so well on a historically difficult ImageNet dataset. Utilizing techniques that are still used today, such as data augmentation and dropout, this paper really illustrated the benefits of CNNs and backed them up with record breaking performance in the competition.

<img src='https://cdn-images-1.medium.com/max/1600/1*qyc21qM0oxWEuRaj-XJKcw.png' style="width:800px;">

In [ ]:
def alexnet():
    return Sequential([
        # 1
        Conv2D(96, (11, 11), strides=4, input_shape=(227,227,3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        BatchNormalization(),
        
        # 2
        ZeroPadding2D((2, 2)),
        Conv2D(256, (5, 5), activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        BatchNormalization(),
        
        # 3
        ZeroPadding2D((1, 1)),
        Conv2D(384, (3, 3), activation='relu'),
        
        # 4
        ZeroPadding2D((1, 1)),
        Conv2D(384, (3, 3), activation='relu'),
        
        # 5
        ZeroPadding2D((1, 1)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        # 6
        Flatten(),
        Dense(4096, activation = 'relu'),
        Dropout(0.5),
        
        # 7
        Dense(4096, activation = 'relu'),
        Dropout(0.5),
        
        # 8
        Dense(num_classes, activation = 'softmax')
    ])


In [ ]:
myModel = alexnet()
myModel.summary()

---
# ZF-Net [2013]

   ZF Net was not only the winner of the competition in 2013, but also provided great intuition as to the workings on CNNs and illustrated more ways to improve performance. The visualization approach described helps not only to explain the inner workings of CNNs, but also provides insight for improvements to network architectures. The fascinating deconv visualization approach and occlusion experiments make this one of my personal favorite papers.

<img src='https://adeshpande3.github.io/assets/zfnet.png' style="width:800px;">

In [ ]:
def zfnet():
    return Sequential([
        
        # First convolutional Layer (96x7x7)
        Conv2D(96, (7,7), input_shape=(224,224,3), strides = 2, activation = "relu"),
        ZeroPadding2D((1,1)),
        MaxPooling2D((3,3), strides=2),
        BatchNormalization(),

        # Second convolutional Layer (256x5x5)
        Conv2D(256, (5,5), strides = 2, activation = "relu"),
        ZeroPadding2D((1,1)),
        MaxPooling2D((3,3), strides=2),
        BatchNormalization(),

        # 3
        ZeroPadding2D((1,1)),
        Conv2D(384, (3,3), activation = "relu"),

        # 4
        ZeroPadding2D((1,1)),
        Conv2D(384, (3,3), activation = "relu"),

        # 5
        ZeroPadding2D((1,1)),
        Conv2D(256, (3,3), activation = "relu"),
        MaxPooling2D((3,3), strides=2),
        
        # 6
        Flatten(),
        Dense(4096, activation="relu"),
        Dropout(0.5),

        # 7
        Dense(4096, activation="relu"),
        Dropout(0.5),
        
        # 8
        Dense(num_classes, activation = 'softmax')
    ])


In [ ]:
myModel = zfnet()
myModel.summary()

---
# VGG-Net [2014]

   VGG Net is one of the most influential papers in my mind because it reinforced the notion that convolutional neural networks have to have a deep network of layers in order for this hierarchical representation of visual data to work. Keep it deep. Keep it simple.

<img src='http://www.sanko-shoko.net/note.php?img=y1kl' style="width:800px;">

In [ ]:
def vgg16():
    return Sequential([
        
        # 1-2
        Conv2D(64, 3, input_shape=(224,224,3), activation='relu', padding='same'),
        Conv2D(64, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 3-4
        Conv2D(128, 3, activation='relu', padding='same'),
        Conv2D(128, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 5-7
        Conv2D(256, 3, activation='relu', padding='same'),
        Conv2D(256, 3, activation='relu', padding='same'),
        Conv2D(256, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        
        # 8-10
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 11-13
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 14
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        
        # 15
        Dense(4096, activation='relu'),
        Dropout(0.5),
        
        # 16
        Dense(num_classes, activation='softmax')
    ])


In [ ]:
myModel = vgg16()
myModel.summary()

**IF DON'T ALREADY HAVE A VGG16 MODEL, <br>THIS CODE WILL DOWNLOAD THE MODEL WITH A FAIRLY LARGE SIZE** 
---

**Do not run the last four lines of code if the connection is slow**

In [ ]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights=None, include_top=True)

# model size ~530MB
model = VGG16(weights='imagenet', include_top=True)
model.summary()

# model size ~57MB
model = VGG16(weights='imagenet', include_top=False)
model.summary()

In [ ]:
def vgg16_GAP():
    return Sequential([
        
        # 1-2
        Conv2D(64, 3, input_shape=(224,224,3), activation='relu', padding='same'),
        Conv2D(64, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 3-4
        Conv2D(128, 3, activation='relu', padding='same'),
        Conv2D(128, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 5-7
        Conv2D(256, 3, activation='relu', padding='same'),
        Conv2D(256, 3, activation='relu', padding='same'),
        Conv2D(256, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        
        # 8-10
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 11-13
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        Conv2D(512, 3, activation='relu', padding='same'),
        MaxPooling2D(2, 2),

        # 14
        GlobalAveragePooling2D(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        
        # 15
        Dense(4096, activation='relu'),
        Dropout(0.5),
        
        # 16
        Dense(num_classes, activation='softmax')
    ])


In [ ]:
myModel = vgg16_GAP()
myModel.summary()

---
# GoogLeNet [2014]

GoogLeNet was one of the first models that introduced the idea that CNN layers didn’t always have to be stacked up sequentially. Coming up with the Inception module, the authors showed that a creative structuring of layers can lead to improved performance and computationally efficiency. This paper has really set the stage for some amazing architectures that we could see in the coming years.

<img src='https://adeshpande3.github.io/assets/GoogleNet.gif' style="width:800px;">
<img src='https://adeshpande3.github.io/assets/GoogLeNet.png' style="width:800px;">

In [ ]:
def small_googlenet():
    
    input_img = Input(shape = (32, 32, 3))

    tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
    tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)

    tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
    tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    
    output = Flatten()(output)
    out    = Dense(10, activation='softmax')(output)

    model = Model(inputs = input_img, outputs = out)
    
    return model


In [ ]:
myModel = small_googlenet()
myModel.summary()

In [ ]:
from keras.applications.inception_v3 import InceptionV3

model = InceptionV3(weights=None, include_top=True)
model.summary()
print('Total params:', format(model.count_params(),',d'))

---
# Residual Network (ResNet) [2015]

3.6% error rate. That itself should be enough to convince you. 
The ResNet model is the best CNN architecture that we currently have and is a great innovation for the idea of residual learning. 

<img src='https://adeshpande3.github.io/assets/ResNet.gif' style="width:300px;">

In [ ]:
def resnet_layer(inputs, num_filters=16, kernel_size=3,
                 strides=1, activation='relu', batch_normalization=True,
                 conv_first=True):
    
    conv = Conv2D(num_filters, kernel_size=kernel_size, strides=strides,
                  padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1():
    input_shape = (32, 32, 3)
    depth = 20
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
                
            y = resnet_layer(inputs=x,
                             num_filters=num_filters, strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters, activation=None)
            
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters, kernel_size=1, strides=strides,
                                 activation=None, batch_normalization=False)
                
                
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
myModel = resnet_v1()
myModel.summary()

In [ ]:
from keras.applications.resnet50 import ResNet50

model = ResNet50(weights=None, include_top=True)
# model.summary()
print('Total params:', format(model.count_params(),',d'))

---
# Going Further: Wide ResNet [2016]


<img src='http://i.imgur.com/3b0fw7b.png' style="width:500px;">

---
# Going Further: DenseNet [2016]


<img src='https://raw.githubusercontent.com/titu1994/DenseNet/master/images/dense_net.JPG' style="width:800px;">

---
# Going Further: ResNet in ResNet [2016]


<img src='https://raw.githubusercontent.com/titu1994/Residual-of-Residual-Networks/master/images/resnet%20vs%20ror.JPG' style="width:400px;">

---
# Going Further: MobileNet [2017]


<img src='https://raw.githubusercontent.com/titu1994/MobileNetworks/master/images/mobilenets.PNG' style="width:800px;">

---
# Going Further: SENet [2017]


<img src='https://raw.githubusercontent.com/titu1994/keras-squeeze-excite-network/master/images/squeeze-excite-block.JPG' style="width:800px;">

---
# Going Further: ResNeXt [2017]


<img src='https://raw.githubusercontent.com/titu1994/Keras-ResNeXt/master/images/Cardinality.PNG' style="width:500px;">

![footer](https://image.ibb.co/hAHDYK/footer2018.png)